In [10]:
import os
import pandas as pd
import raws_sigs as raws
import numpy as np
from datetime import datetime, timedelta, UTC

In [11]:
gacc_region = 'oscc'
gacc_region = gacc_region.upper()

try:
    now = datetime.now(UTC)
except Exception as e:
    now = datetime.utcnow()
    
last_year = now.year - 1

if now.month >= 11:
    start_date = datetime(now.year, 11, 1)
else:
    start_date = datetime(last_year, 11, 1)
end_date = datetime(now.year, now.month, now.day)

paths = []
for folder in os.listdir(f"FEMS Data/Stations/{gacc_region}"):
    path = f"FEMS Data/Stations/{gacc_region}/{folder}"
    paths.append(path)

psa = 1
for p in range(0, len(paths)):
    files = []
    for file in os.listdir(paths[p]):
        files.append(file)

    dates = []
    
    dfm_100 = []
    dfm_100hr_mean = []
    dfm_100hr_min = []
    dfm_100hr_max = []
    
    dfm_1000 = []
    dfm_1000hr_mean = []
    dfm_1000hr_min = []
    dfm_1000hr_max = []

    erc = []
    erc_mean = []
    erc_min = []
    erc_max = []
    
    for i in range(0, len(files)):
        try:
            df = pd.read_csv(f"{paths[p]}/{files[i]}")
            df = df[df['observationTime'].between(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))]
            dfm_100.append(df['hundredHR_TL_FuelMoisture'])
            dates.append(df['observationTime'])
            dfm_1000.append(df['thousandHR_TL_FuelMoisture'])
            erc.append(df['energyReleaseComponent'])
            
        except Exception as e:
            pass

    try:
        df_dates = pd.DataFrame(dates)
    except Exception as e:
        pass

    try:
        df_100 = pd.DataFrame(dfm_100)
        df_100 = df_100.transpose()
        for i in range(0, len(df_100)):
            mean = np.nanmean(df_100.iloc[i])
            dfm_100hr_mean.append(mean)
    except Exception as e:
        pass

    try:
        for i in range(0, len(df_100)):
            minimum = np.nanmin(df_100.iloc[i])
            dfm_100hr_min.append(minimum)
    except Exception as e:
        pass        

    try:
        for i in range(0, len(df_100)):
            maximum = np.nanmax(df_100.iloc[i])
            dfm_100hr_max.append(maximum)
    except Exception as e:
        pass    

    try:
        df_1000 = pd.DataFrame(dfm_1000)
        df_1000 = df_1000.transpose()
        for i in range(0, len(df_1000)):
            mean = np.nanmean(df_1000.iloc[i])
            dfm_1000hr_mean.append(mean)
    except Exception as e:
        pass

    try:
        for i in range(0, len(df_1000)):
            minimum = np.nanmin(df_1000.iloc[i])
            dfm_1000hr_min.append(minimum)
    except Exception as e:
        pass        

    try:
        for i in range(0, len(df_1000)):
            maximum = np.nanmax(df_1000.iloc[i])
            dfm_1000hr_max.append(maximum)
    except Exception as e:
        pass    

    try:
        df_erc = pd.DataFrame(erc)
        df_erc = df_erc.transpose()
        for i in range(0, len(df_erc)):
            mean = np.nanmean(df_erc.iloc[i])
            erc_mean.append(mean)
    except Exception as e:
        pass

    try:
        for i in range(0, len(df_erc)):
            minimum = np.nanmin(df_erc.iloc[i])
            erc_min.append(minimum)
    except Exception as e:
        pass        

    try:
        for i in range(0, len(df_erc)):
            maximum = np.nanmax(df_erc.iloc[i])
            erc_max.append(maximum)
    except Exception as e:
        pass    


    main = pd.DataFrame()
    
    main['date'] = df_dates.iloc[0]
    
    main['100hr_DFM_mean'] = dfm_100hr_mean
    main['100hr_DFM_min'] = dfm_100hr_min
    main['100hr_DFM_max'] = dfm_100hr_max

    main['1000hr_DFM_mean'] = dfm_1000hr_mean
    main['1000hr_DFM_min'] = dfm_1000hr_min
    main['1000hr_DFM_max'] = dfm_1000hr_max

    main['ERC_mean'] = erc_mean
    main['ERC_min'] = erc_min
    main['ERC_max'] = erc_max

    if os.path.exists(f"FEMS Data/{gacc_region}/PSA Data"):
        pass
    else:
        os.mkdir(f"FEMS Data/{gacc_region}/PSA Data")
    
    fname = f"zone_{psa}.csv"
    main.to_csv(fname, index=False)
    os.replace(f"{fname}", f"FEMS Data/{gacc_region}/PSA Data/{fname}")
    psa = psa + 1

In [16]:
main

,date,100hr_DFM_mean,100hr_DFM_min,100hr_DFM_max,1000hr_DFM_mean,1000hr_DFM_min,1000hr_DFM_max,ERC_mean,ERC_min,ERC_max
5783,2024-11-01,9.150000,8.46,10.30,8.880000,8.04,10.08,55.970000,50.45,59.53
5784,2024-11-02,9.423333,8.78,10.52,8.890000,8.07,10.07,56.120000,51.18,59.27
5785,2024-11-03,9.640000,9.02,10.67,8.930000,8.11,10.11,54.103333,48.02,57.86
5786,2024-11-04,9.820000,9.23,10.82,8.966667,8.15,10.14,55.590000,50.24,59.29
5787,2024-11-05,10.000000,9.43,11.01,9.033333,8.23,10.20,54.673333,49.18,58.18
...,...,...,...,...,...,...,...,...,...,...
5925,2025-03-23,10.093333,9.75,10.71,12.090000,11.28,13.10,46.516667,43.42,48.93
5926,2025-03-24,11.050000,10.57,11.70,12.066667,11.37,13.02,48.016667,44.45,50.76
5927,2025-03-25,10.473333,9.67,11.51,11.913333,11.36,12.89,49.293333,44.57,51.74
5928,2025-03-26,9.890000,8.99,11.24,11.636667,11.08,12.74,50.650000,45.66,53.42
